In [3]:
# 💰 Personal Expense Tracker — PoC

#  **Objective:** Build a compact,  expense tracker in Jupyter that demonstrates Core Python, OOP, file handling, logging, error handling, generators and a simple visualization.

#  **How to use:** Run cells in order. The notebook will create sample data, define the `ExpenseTracker` class, and show example usage.

In [9]:
import csv  # for reading/writing expense files.
import os   # for directory handling.
import datetime  #  for working with dates.
import logging  # to track program activities or errors
import matplotlib.pyplot as plt  # for data visualization.

# Create folders for data and logs  if they don’t already exist.
os.makedirs('Data', exist_ok = True)
os.makedirs('log', exist_ok = True)


# Setup logging
logging.basicConfig(
    filename = 'log/expense_tracker.log',
    level = logging.INFO,
    format = '%(asctime)s - %(levelname)s - %(message)s'   # 2025-10-30 14:10:22,658 - INFO - Expense added successfully.  current timestamp - INFO, WARNING, ERROR, etc.-your log message
)

print("✅ Setup complete. Directories: data/, logs/")

✅ Setup complete. Directories: data/, logs/


In [ ]:
# Helper Decorator

def log_time(fun):
    def wrapper(*args,**kwargs):
        start = time.time()
        result = fun(*arga, **kwargs)
        end = time.time()
        print(f"{fun.__name__} Executed in {end - start:.3f}s")
        return result
    return wrapper

print("✅ Decorator ready")

In [10]:
# Create Sample CSV Data

sample_path = 'data/expenses_sample.csv'

if not os.path.exists(sample_path):
    rows = [
        {"date":"2025-10-01","category":"Food","amount":"150"},
        {"date":"2025-10-02","category":"Travel","amount":"60"},
        {"date":"2025-10-03","category":"Utilities","amount":"1200"},
        {"date":"2025-10-05","category":"Food","amount":"200"},
        {"date":"2025-09-28","category":"Shopping","amount":"800"},
        {"date":"2025-09-30","category":"Food","amount":"90"},
    ]
    with open(sample_path, 'w', newline='', encoding='utf-8') as f:
        writer  = csv.DictWriter(f, fieldnames=['date','category','amount'])
        writer.writeheader()
        writer.writerows(rows)
    print(f"✅ Sample CSV created at: {sample_path}")
else:
    print(f"ℹ️ Sample CSV already exists at: {sample_path}")
        

✅ Sample CSV created at: data/expenses_sample.csv
